# Problem Description
Customer churn, the process by which customers stop usigng a company's offerings, is a critical challenge for businesses, especially in competetive industries like telecommunications. These factors that influence customer churn are essential for streamlining operations, reducing costs, and improving user experience.

The Telco Customer Churn dataset provides a vast set of data about a fictional telecommunications company's customers, including demographics, account information, and usage paterns. Each record is labeled with a binary classification of whether the customer has churned or not.

A solution to this problem will greatly alleviate the burden upon companies of dumping resources upon customers who will churn over and will be helpful for reducing unpredictability and cost. The primary stages of solving this problem are as follows:

1. Exploratory Data Analysis
2. Model Architecture and Training
3. Results and Performance
4. Discussion and Conclusion

# Exploratory Data Analysis
First, we will load the [data](https://www.kaggle.com/datasets/blastchar/telco-customer-churn) and display the schema

In [38]:
import pandas as pd

In [39]:
df = pd.read_csv('TelcoCustomerChurn.csv')

In [40]:
# Dataset Structure
df.info()

# Dataset Summary
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL             No  ...   
1             Yes                No             DSL            Yes  ...   
2             Yes                No             DSL            Yes  ...   
3              No  No phone service             DSL            Yes  ...   
4             Yes                No     Fiber optic             No  ...   
...           ...               ...             ...            ...  ...   
7038          Yes               Yes             DSL            Yes  ...   
7039          Yes               Yes     Fiber optic             No  ...   
7040           No  No phone service             DSL            Yes  ...   
7041          Yes               Yes     Fiber optic             No  ...   
7042          Yes                No     Fiber optic            Yes  ...   

     DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
0                  No          No          No              No  Month-to-month   
1                 Yes          No          No              No        One year   
2                  No          No          No              No  Month-to-month   
3                 Yes         Yes          No              No        One year   
4                  No          No          No              No  Month-to-month   
...               ...         ...         ...             ...             ...   
7038              Yes         Yes         Yes             Yes        One year   
7039              Yes          No         Yes             Yes        One year   
7040               No          No          No              No  Month-to-month   
7041               No          No          No              No  Month-to-month   
7042              Yes         Yes         Yes             Yes        Two year   

     PaperlessBilling              PaymentMethod MonthlyCharges  TotalCharges  \
0                 Yes           Electronic check          29.85         29.85   
1                  No               Mailed check          56.95        1889.5   
2                 Yes               Mailed check          53.85        108.15   
3                  No  Bank transfer (automatic)          42.30       1840.75   
4                 Yes           Electronic check          70.70        151.65   
...               ...                        ...            ...           ...   
7038              Yes               Mailed check          84.80        1990.5   
7039              Yes    Credit card (automatic)         103.20        7362.9   
7040              Yes           Electronic check          29.60        346.45   
7041              Yes               Mailed check          74.40         306.6   
7042              Yes  Bank transfer (automatic)         105.65        6844.5   

     Churn  
0       No  
1       No  
2      Yes  
3       No  
4      Yes  
...    ...  
7038    No  
7039    No  
7040    No  
7041   Yes  
7042    No  

[7043 rows x 21 columns]

## Predictors and Feature
As can be seen, the dataset consists of 20 predictors and 1 feature (Churn). This machine learning problem is in the class of binary classification problems - the feature is a discrete variable that can only take two values, corresponding to "Yes" (the customer did churn) or "No" (the customer did not churn). This dataset is known to have no null values and no duplicate records, so we can skip that step. Now, we will look at some statistical properties of the dataset.

In [41]:
df.describe()

SeniorCitizen       tenure  MonthlyCharges
count    7043.000000  7043.000000     7043.000000
mean        0.162147    32.371149       64.761692
std         0.368612    24.559481       30.090047
min         0.000000     0.000000       18.250000
25%         0.000000     9.000000       35.500000
50%         0.000000    29.000000       70.350000
75%         0.000000    55.000000       89.850000
max         1.000000    72.000000      118.750000

The fictional company's monthly charges to users ranges from  \\$18.25 to \\$118.75 with the tenure (the number of months that the customer has had their service with the company) ranging from 0 months to 72 months. Below, we delete the customerID column since it is not needed for this model.

In [42]:
df.drop('customerID', axis=1, inplace=True)

While some machine learning models are able to deal with natural language, the model we use in this problem is not readily capable of doing so. Additionally, when the natural languages simply encodes a discrete variable, it is better to numerically encode that natural language into the discrete variable. Therefore, we will enocde the predictors that take on value "Yes" and "No" to 1 and 0, respectively.

In [43]:
yes_no_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for column in yes_no_columns:
    df[column] = df[column].map({
        'Yes': 1,
        'No': 0
    })

Additionally, we will encode the gender column taking on values "Male" and "Female" as 1 and 0, respectively.

In [44]:
df['gender'] = df['gender'].map({
    'Male': 1,
    'Female': 0
})

The Contract column contains ordinal information and can thus be encoded with ordered integers